In [1]:
pip install pymupdf pandas pillow transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
!pip install pymupdf

In [8]:
import torch

# Vérifier si le GPU est disponible
if torch.cuda.is_available():
    device = torch.device("cuda")  # Utiliser le GPU
    print(f"GPU disponible : {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Utiliser le CPU
    print("GPU non disponible, utilisation du CPU.")

GPU disponible : Tesla T4


In [11]:
import fitz  # PyMuPDF
import pandas as pd
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import requests
from transformers.utils import logging

# Configurer un délai d'attente plus long
logging.set_verbosity_warning()  # Réduire les logs pour plus de clarté
requests.adapters.DEFAULT_TIMEOUT = 60  # 60 secondes de délai d'attente

# Fonction pour générer une description d'image
def generate_image_caption(image_path):
    try:
        # Ouvrir l'image
        image = Image.open(image_path).convert("RGB")

        # Prétraiter l'image et générer la description
        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs)

        # Décoder la description
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Erreur lors de la génération de la description pour {image_path}: {e}")
        return "Description non disponible"

# Fonction pour extraire les images et les labelliser
def extract_and_label_images(pdf_path, output_folder):
    # Vérifier si le fichier PDF existe
    if not os.path.exists(pdf_path):
        print(f"Erreur : Le fichier PDF {pdf_path} n'existe pas.")
        return None

    # Ouvrir le PDF
    try:
        pdf_document = fitz.open(pdf_path)
        print(f"PDF ouvert avec succès. Nombre de pages : {len(pdf_document)}")
    except Exception as e:
        print(f"Erreur lors de l'ouverture du PDF : {e}")
        return None

    # Créer un dossier pour sauvegarder les images
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Dossier de sortie créé : {output_folder}")

    # Initialiser une liste pour stocker les données
    data = []

    # Parcourir les pages du PDF
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        print(f"Traitement de la page {page_num + 1}...")

        # Extraire les images de la page
        image_list = page.get_images(full=True)
        print(f"Nombre d'images trouvées sur la page {page_num + 1} : {len(image_list)}")

        for img_index, img in enumerate(image_list):
            xref = img[0]  # Référence de l'image
            try:
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                # Sauvegarder l'image
                image_path = os.path.join(output_folder, f"page_{page_num + 1}_img_{img_index + 1}.{image_ext}")
                with open(image_path, "wb") as image_file:
                    image_file.write(image_bytes)
                print(f"Image sauvegardée : {image_path}")

                # Générer une description automatique
                description = generate_image_caption(image_path)
                print(f"Description générée : {description}")

                # Ajouter les données à la liste
                data.append({
                    "image_path": image_path,
                    "page_number": page_num + 1,
                    "image_index": img_index + 1,
                    "description": description.strip()
                })
            except Exception as e:
                print(f"Erreur lors de l'extraction de l'image {img_index + 1} sur la page {page_num + 1}: {e}")

    # Créer un DataFrame pandas
    df = pd.DataFrame(data)

    # Sauvegarder la base de données en CSV
    csv_path = os.path.join(output_folder, "image_database.csv")
    df.to_csv(csv_path, index=False)

    print(f"Base de données sauvegardée dans {csv_path}")
    return df

# Charger le modèle (en ligne ou localement)
try:
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")
    print("Veuillez vérifier votre connexion Internet ou utiliser un modèle local.")
    exit()

# Utilisation
pdf_path = r"Volume II catalogue_removed.pdf"  # Chemin absolu vers le PDF
output_folder = r"output_images"  # Dossier pour sauvegarder les images
database = extract_and_label_images(pdf_path, output_folder)

# Afficher la base de données
if database is not None:
    print(database.head())
else:
    print("Aucune donnée à afficher.")

PDF ouvert avec succès. Nombre de pages : 384
Dossier de sortie créé : output_images
Traitement de la page 1...
Nombre d'images trouvées sur la page 1 : 0
Traitement de la page 2...
Nombre d'images trouvées sur la page 2 : 0
Traitement de la page 3...
Nombre d'images trouvées sur la page 3 : 0
Traitement de la page 4...
Nombre d'images trouvées sur la page 4 : 0
Traitement de la page 5...
Nombre d'images trouvées sur la page 5 : 5
Image sauvegardée : output_images/page_5_img_1.jpeg
Description générée : a black and white photo of a man in a field
Image sauvegardée : output_images/page_5_img_2.jpeg
Description générée : a black and white photo of a rock in the grass
Image sauvegardée : output_images/page_5_img_3.jpeg
Description générée : a black and white photo of a rocky landscape
Image sauvegardée : output_images/page_5_img_4.jpeg
Description générée : a black and white photo of a field
Image sauvegardée : output_images/page_5_img_5.jpeg
Description générée : a black and white photo 

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Description générée : a rock in the middle of a field
Traitement de la page 15...
Nombre d'images trouvées sur la page 15 : 2900
Image sauvegardée : output_images/page_15_img_1.png
Erreur lors de la génération de la description pour output_images/page_15_img_1.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_2.png
Erreur lors de la génération de la description pour output_images/page_15_img_2.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_3.png
Erreur lors de la génération de la description pour output_images/page_15_img_3.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_4.png
Erreur lors de la génération de la description pour output_images/page_15_img_4.png: mean must have

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Erreur lors de la génération de la description pour output_images/page_15_img_23.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_24.png
Erreur lors de la génération de la description pour output_images/page_15_img_24.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_25.png
Erreur lors de la génération de la description pour output_images/page_15_img_25.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_26.png
Erreur lors de la génération de la description pour output_images/page_15_img_26.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_27.png
Erreur lors de la génération de 

...

The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.


Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_623.png
Erreur lors de la génération de la description pour output_images/page_15_img_623.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_624.png
Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_625.png
Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_626.png
Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_627.png
Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_628.png
Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_629.png
Description généré

The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.


Description générée : a black background with a white and red flower
Image sauvegardée : output_images/page_15_img_634.png
Erreur lors de la génération de la description pour output_images/page_15_img_634.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images/page_15_img_635.png


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import fitz  # PyMuPDF
import pandas as pd
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Vérifier si le GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

# Charger le modèle et le déplacer vers le GPU
try:
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")
    exit()

# Fonction pour générer une description d'image
def generate_image_caption(image_path):
    try:
        # Ouvrir l'image
        image = Image.open(image_path).convert("RGB")

        # Prétraiter l'image et générer la description
        inputs = processor(image, return_tensors="pt").to(device)  # Déplacer les données vers le GPU
        out = model.generate(**inputs)

        # Décoder la description
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Erreur lors de la génération de la description pour {image_path}: {e}")
        return "Description non disponible"

# Fonction pour extraire les images et les labelliser
def extract_and_label_images(pdf_path, output_folder, max_images_per_page=100):
    # Vérifier si le fichier PDF existe
    if not os.path.exists(pdf_path):
        print(f"Erreur : Le fichier PDF {pdf_path} n'existe pas.")
        return None

    # Ouvrir le PDF
    try:
        pdf_document = fitz.open(pdf_path)
        print(f"PDF ouvert avec succès. Nombre de pages : {len(pdf_document)}")
    except Exception as e:
        print(f"Erreur lors de l'ouverture du PDF : {e}")
        return None

    # Créer un dossier pour sauvegarder les images
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Dossier de sortie créé : {output_folder}")

    # Initialiser une liste pour stocker les données
    data = []

    # Parcourir les pages du PDF
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        print(f"Traitement de la page {page_num + 1}...")

        # Extraire les images de la page
        image_list = page.get_images(full=True)
        print(f"Nombre d'images trouvées sur la page {page_num + 1} : {len(image_list)}")

        # Limiter le nombre d'images traitées par page
        if len(image_list) > max_images_per_page:
            print(f"Attention : La page {page_num + 1} contient trop d'images ({len(image_list)}). Seules les premières {max_images_per_page} images seront traitées.")
            image_list = image_list[:max_images_per_page]

        for img_index, img in enumerate(image_list):
            xref = img[0]  # Référence de l'image
            try:
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                # Sauvegarder l'image
                image_path = os.path.join(output_folder, f"page_{page_num + 1}_img_{img_index + 1}.{image_ext}")
                with open(image_path, "wb") as image_file:
                    image_file.write(image_bytes)
                print(f"Image sauvegardée : {image_path}")

                # Générer une description automatique
                description = generate_image_caption(image_path)
                print(f"Description générée : {description}")

                # Ajouter les données à la liste
                data.append({
                    "image_path": image_path,
                    "page_number": page_num + 1,
                    "image_index": img_index + 1,
                    "description": description.strip()
                })
            except Exception as e:
                print(f"Erreur lors de l'extraction de l'image {img_index + 1} sur la page {page_num + 1}: {e}")

    # Créer un DataFrame pandas
    df = pd.DataFrame(data)

    # Sauvegarder la base de données en CSV
    csv_path = os.path.join(output_folder, "image_database.csv")
    df.to_csv(csv_path, index=False)

    print(f"Base de données sauvegardée dans {csv_path}")
    return df

# Utilisation
pdf_path = r"Volume II catalogue_removed.pdf"  # Chemin absolu vers le PDF
output_folder = r"output_images1"  # Dossier pour sauvegarder les images
database = extract_and_label_images(pdf_path, output_folder)

# Afficher la base de données
if database is not None:
    print(database.head())
else:
    print("Aucune donnée à afficher.")

Utilisation de : cuda
PDF ouvert avec succès. Nombre de pages : 384
Dossier de sortie créé : output_images1
Traitement de la page 1...
Nombre d'images trouvées sur la page 1 : 0
Traitement de la page 2...
Nombre d'images trouvées sur la page 2 : 0
Traitement de la page 3...
Nombre d'images trouvées sur la page 3 : 0
Traitement de la page 4...
Nombre d'images trouvées sur la page 4 : 0
Traitement de la page 5...
Nombre d'images trouvées sur la page 5 : 5
Image sauvegardée : output_images1/page_5_img_1.jpeg
Description générée : a black and white photo of a man in a field
Image sauvegardée : output_images1/page_5_img_2.jpeg
Description générée : a black and white photo of a rock in the grass
Image sauvegardée : output_images1/page_5_img_3.jpeg
Description générée : a black and white photo of a rocky landscape
Image sauvegardée : output_images1/page_5_img_4.jpeg
Description générée : a black and white photo of a field
Image sauvegardée : output_images1/page_5_img_5.jpeg
Description généré

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Description générée : a rock in the middle of a field
Traitement de la page 15...
Nombre d'images trouvées sur la page 15 : 2900
Attention : La page 15 contient trop d'images (2900). Seules les premières 100 images seront traitées.
Image sauvegardée : output_images1/page_15_img_1.png
Erreur lors de la génération de la description pour output_images1/page_15_img_1.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15_img_2.png
Erreur lors de la génération de la description pour output_images1/page_15_img_2.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15_img_3.png
Erreur lors de la génération de la description pour output_images1/page_15_img_3.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Image sauvegardée : output_images1/page_15_img_23.png
Erreur lors de la génération de la description pour output_images1/page_15_img_23.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15_img_24.png
Erreur lors de la génération de la description pour output_images1/page_15_img_24.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15_img_25.png
Erreur lors de la génération de la description pour output_images1/page_15_img_25.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_15_img_26.png
Erreur lors de la génération de la description pour output_images1/page_15_img_26.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : outp

...

Erreur lors de la génération de la description pour output_images1/page_125_img_93.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_125_img_94.png
Erreur lors de la génération de la description pour output_images1/page_125_img_94.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_125_img_95.png
Erreur lors de la génération de la description pour output_images1/page_125_img_95.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_125_img_96.png
Erreur lors de la génération de la description pour output_images1/page_125_img_96.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_125_img_97.png
Erreur lors de l

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Erreur lors de la génération de la description pour output_images1/page_126_img_14.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_15.png
Erreur lors de la génération de la description pour output_images1/page_126_img_15.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_16.png
Erreur lors de la génération de la description pour output_images1/page_126_img_16.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_17.png
Erreur lors de la génération de la description pour output_images1/page_126_img_17.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_18.png
Erreur lors de l

The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 2, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image 

Erreur lors de la génération de la description pour output_images1/page_126_img_62.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_63.png
Erreur lors de la génération de la description pour output_images1/page_126_img_63.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_64.png
Erreur lors de la génération de la description pour output_images1/page_126_img_64.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_65.png
Erreur lors de la génération de la description pour output_images1/page_126_img_65.png: mean must have 1 elements if it is an iterable, got 3
Description générée : Description non disponible
Image sauvegardée : output_images1/page_126_img_66.png
Erreur lors de l

In [17]:
import shutil

shutil.make_archive("images_exportées", 'zip', "output_images1")
from google.colab import files
files.download("images_exportées.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>